# Chapter 20: Integration and Final Projects

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_20_integration_and_projects.ipynb)

**Integration and projects** bring together all techniques from the previous chapters into complete rendering systems. This final chapter guides you through building production-quality renderers and provides project ideas for further exploration.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Tuple, List, Optional

## 1. System Architecture

Building a complete rendering system requires careful integration of all components.

### Renderer Architecture Patterns

**1. Scene Graph**
```
Scene
├── Camera
├── Lights[]
├── Materials[]
├── Meshes[]
│   ├── Geometry
│   ├── Material
│   └── Transform
└── Acceleration Structure (BVH)
```

**2. Render Pipeline**
```
Input (Scene + Camera)
   ↓
Geometry Processing
   ↓
Acceleration Structure Build
   ↓
Rendering (Rasterize or Ray Trace)
   ↓
Post-Processing
   ↓
Output (Image)
```

**3. Component Relationships**

- **Scene** owns all objects and manages lifetime
- **Camera** defines view transformation
- **Materials** describe surface properties (BRDF)
- **Lights** provide illumination
- **Acceleration structures** enable fast ray queries
- **Framebuffer** stores rendered output

### Design Principles

✅ **Modularity**: Separate concerns (geometry, shading, acceleration)
✅ **Extensibility**: Easy to add new primitives, materials, lights
✅ **Performance**: Cache-friendly data structures
✅ **Testability**: Unit test individual components
✅ **Configurability**: JSON/YAML scene files

In [ ]:
from dataclasses import dataclass, field
from enum import Enum
from typing import Dict, Any
import json

# ============================================================================
# COMPLETE INTEGRATED RENDERER SYSTEM
# ============================================================================

# Vector and Matrix classes (simplified versions from Chapter 1)
class Vec3:
    """3D Vector"""
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = x
        self.y = y
        self.z = z
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self):
        return math.sqrt(self.dot(self))
    
    def normalize(self):
        l = self.length()
        if l > 0:
            return Vec3(self.x / l, self.y / l, self.z / l)
        return Vec3(0, 0, 0)
    
    def to_array(self):
        return np.array([self.x, self.y, self.z])

@dataclass
class Ray:
    """Ray with origin and direction"""
    origin: Vec3
    direction: Vec3
    
    def at(self, t: float) -> Vec3:
        """Get point at parameter t"""
        return self.origin + self.direction * t

@dataclass
class HitRecord:
    """Intersection record"""
    t: float = float('inf')
    point: Vec3 = field(default_factory=Vec3)
    normal: Vec3 = field(default_factory=Vec3)
    material_id: int = -1
    hit: bool = False

class MaterialType(Enum):
    """Material types"""
    LAMBERTIAN = 1
    METAL = 2
    DIELECTRIC = 3
    PBR = 4

@dataclass
class Material:
    """Material properties"""
    type: MaterialType
    albedo: Vec3 = field(default_factory=lambda: Vec3(0.8, 0.8, 0.8))
    roughness: float = 0.5
    metallic: float = 0.0
    ior: float = 1.5
    emission: Vec3 = field(default_factory=Vec3)

class Sphere:
    """Sphere primitive"""
    def __init__(self, center: Vec3, radius: float, material_id: int):
        self.center = center
        self.radius = radius
        self.material_id = material_id
    
    def intersect(self, ray: Ray) -> HitRecord:
        """Ray-sphere intersection"""
        oc = ray.origin - self.center
        a = ray.direction.dot(ray.direction)
        half_b = oc.dot(ray.direction)
        c = oc.dot(oc) - self.radius * self.radius
        
        discriminant = half_b * half_b - a * c
        
        hit = HitRecord()
        
        if discriminant < 0:
            return hit
        
        sqrtd = math.sqrt(discriminant)
        root = (-half_b - sqrtd) / a
        
        if root < 0.001 or root > 1000:
            root = (-half_b + sqrtd) / a
            if root < 0.001 or root > 1000:
                return hit
        
        hit.t = root
        hit.point = ray.at(root)
        hit.normal = (hit.point - self.center) * (1.0 / self.radius)
        hit.material_id = self.material_id
        hit.hit = True
        
        return hit

@dataclass
class Light:
    """Light source"""
    position: Vec3
    color: Vec3
    intensity: float = 1.0

class Camera:
    """Pinhole camera"""
    def __init__(self, look_from: Vec3, look_at: Vec3, vup: Vec3, vfov: float, aspect: float):
        theta = vfov * math.pi / 180.0
        h = math.tan(theta / 2.0)
        viewport_height = 2.0 * h
        viewport_width = aspect * viewport_height
        
        w = (look_from - look_at).normalize()
        u = vup.cross(w).normalize()
        v = w.cross(u)
        
        self.origin = look_from
        self.horizontal = u * viewport_width
        self.vertical = v * viewport_height
        self.lower_left = self.origin - self.horizontal * 0.5 - self.vertical * 0.5 - w
    
    def get_ray(self, u: float, v: float) -> Ray:
        """Get ray through (u, v) on viewport"""
        direction = (self.lower_left + self.horizontal * u + self.vertical * v - self.origin).normalize()
        return Ray(self.origin, direction)

class Scene:
    """Complete scene representation"""
    def __init__(self):
        self.objects: List[Sphere] = []
        self.materials: List[Material] = []
        self.lights: List[Light] = []
        self.camera: Optional[Camera] = None
        self.background = Vec3(0.5, 0.7, 1.0)  # Sky color
    
    def add_sphere(self, center: Vec3, radius: float, material: Material) -> int:
        """Add sphere to scene"""
        mat_id = len(self.materials)
        self.materials.append(material)
        self.objects.append(Sphere(center, radius, mat_id))
        return len(self.objects) - 1
    
    def add_light(self, position: Vec3, color: Vec3, intensity: float = 1.0):
        """Add light to scene"""
        self.lights.append(Light(position, color, intensity))
    
    def set_camera(self, camera: Camera):
        """Set scene camera"""
        self.camera = camera
    
    def intersect(self, ray: Ray) -> HitRecord:
        """Find closest intersection"""
        closest = HitRecord()
        
        for obj in self.objects:
            hit = obj.intersect(ray)
            if hit.hit and hit.t < closest.t:
                closest = hit
        
        return closest

print("✓ Core rendering system classes loaded")

## 6. Course Summary and Next Steps

Congratulations! You've completed all 20 chapters of this comprehensive computer graphics tutorial.

### What You've Learned

**Foundations (Chapters 1-4)**
- ✅ Vector and matrix mathematics
- ✅ Quaternions and rotations
- ✅ 2D rasterization algorithms
- ✅ 3D geometry and transformations
- ✅ Projection and viewing systems

**Core Rendering (Chapters 5-9)**
- ✅ Visibility determination and culling
- ✅ Triangle rasterization and barycentric coordinates
- ✅ Phong and Blinn-Phong shading
- ✅ Ray tracing fundamentals
- ✅ Acceleration structures (BVH, k-d tree)

**Advanced Rendering (Chapters 10-14)**
- ✅ Distribution ray tracing (DOF, motion blur, soft shadows)
- ✅ Path tracing and global illumination
- ✅ Physically based rendering (PBR)
- ✅ Texturing and filtering
- ✅ Advanced effects (AO, SSS, volumetrics)

**Specialized Topics (Chapters 15-20)**
- ✅ Animation and physics simulation
- ✅ Geometric algorithms and mesh processing
- ✅ Production rendering techniques
- ✅ GPU optimization and parallelization
- ✅ Real-time ray tracing and denoising
- ✅ System integration

### Key Algorithms Mastered

| Category | Algorithms |
|----------|------------|
| **Math** | Vector ops, matrix multiplication, quaternions, SLERP |
| **2D Graphics** | Bresenham line, circle rasterization, clipping |
| **3D Geometry** | Transformations, projections, Bézier curves |
| **Visibility** | Z-buffer, BSP tree, frustum culling, back-face culling |
| **Rasterization** | Scanline, barycentric, edge functions |
| **Shading** | Phong, Blinn-Phong, Cook-Torrance, Disney BRDF |
| **Ray Tracing** | Ray-sphere, ray-triangle, Möller-Trumbore |
| **Acceleration** | BVH (SAH), k-d tree, octree, grid |
| **Sampling** | Monte Carlo, importance sampling, MIS, stratified |
| **Global Illumination** | Path tracing, photon mapping, NEE |
| **Texturing** | Bilinear, trilinear, mipmaps, anisotropic filtering |
| **Animation** | Keyframes, IK, skinning, physics integration |
| **Mesh Processing** | Subdivision, simplification, convex hull |
| **Optimization** | Multi-threading, SIMD, CUDA, cache optimization |
| **Real-Time** | Denoising, temporal accumulation, ReSTIR |

### Resources for Continued Learning

**Essential Books**
1. **Physically Based Rendering** (Pharr, Jakob, Humphreys)
   - The bible of offline rendering
   - Complete path tracer implementation
   - Theory + practice

2. **Real-Time Rendering** (Akenine-Möller et al.)
   - Comprehensive real-time graphics
   - Game engine techniques
   - Modern GPU pipelines

3. **Computer Graphics: Principles and Practice** (Hughes et al.)
   - Broad foundation
   - Mathematical rigor
   - Classic algorithms

4. **Ray Tracing in One Weekend** series (Shirley)
   - Practical, hands-on
   - Quick results
   - Great for beginners

**Online Resources**
- **SIGGRAPH**: Annual conference papers and courses
- **Shadertoy**: Community shader experiments
- **Scratchapixel**: Free online graphics tutorials
- **Learn OpenGL**: Modern OpenGL tutorials
- **GPU Gems** series: NVIDIA's technique books (free online)

**Communities**
- r/GraphicsProgramming (Reddit)
- Graphics Programming Discord servers
- Shadertoy community
- SIGGRAPH conferences

### Career Paths in Computer Graphics

**Game Development**
- Graphics Programmer
- Engine Developer
- Technical Artist
- Rendering Engineer
- VFX Programmer

**Film & VFX**
- Lighting TD
- Rendering Engineer
- Pipeline TD
- Look Development Artist
- FX Technical Director

**Real-Time Graphics**
- Engine Developer (Unity, Unreal)
- AR/VR Graphics Engineer
- WebGL/WebGPU Developer
- Mobile Graphics Optimization

**Research & Development**
- Graphics Researcher (Academia)
- R&D Engineer (NVIDIA, AMD, Intel)
- Rendering Scientist (Pixar, Disney, ILM)
- Computer Vision Researcher

**Emerging Fields**
- Neural Rendering
- AI-Assisted Graphics
- Cloud Rendering
- Real-Time Ray Tracing
- Metaverse/Virtual Worlds

### Technologies to Explore Next

**Graphics APIs**
- **OpenGL**: Widely supported, good for learning
- **Vulkan**: Modern, explicit control, high performance
- **DirectX 12**: Windows/Xbox, similar to Vulkan
- **Metal**: macOS/iOS, Apple's API
- **WebGPU**: Next-gen web graphics

**GPU Computing**
- **CUDA**: NVIDIA GPU programming
- **OptiX**: NVIDIA ray tracing
- **OpenCL**: Cross-platform GPU compute
- **Compute Shaders**: GPU compute in graphics APIs

**Game Engines**
- **Unity**: C#, widely used, good documentation
- **Unreal Engine**: C++, AAA quality, free for learning
- **Godot**: Open source, beginner-friendly

**Specialized Tools**
- **Blender**: Open source 3D creation suite
- **Houdini**: Procedural generation and VFX
- **RenderMan**: Production renderer
- **Arnold**: Physically based renderer

### Final Thoughts

You've built a strong foundation in computer graphics—from basic math to advanced rendering. The journey from theory to practice is continuous:

**Keep Building**: The best way to learn is by doing. Start with a small project and expand it.

**Read Code**: Study production renderers (pbrt, Mitsuba) to see professional implementations.

**Stay Current**: Graphics evolves rapidly. Follow SIGGRAPH, GPU companies, and game studios for latest techniques.

**Join Communities**: Share your work, ask questions, help others. The graphics community is welcoming!

**Experiment**: Try crazy ideas. Break things. Graphics is where math meets art—creativity is essential.

### You're Ready!

You now have the knowledge to:
- ✅ Build a software rasterizer from scratch
- ✅ Implement a physically based path tracer
- ✅ Understand modern game engine rendering
- ✅ Read and implement research papers
- ✅ Optimize for both CPU and GPU
- ✅ Create production-quality renders

The field of computer graphics is vast and exciting. Whether you're interested in games, films, research, or emerging technologies like VR and neural rendering, you have the foundation to succeed.

**Your graphics journey has just begun. Happy rendering!** 🎨✨

---

### Acknowledgments

This 20-chapter tutorial covers fundamental to advanced topics in 3D computer graphics, with all code written from scratch in Python for educational purposes.

Special thanks to the graphics community and the countless researchers who've advanced this field.

For questions, contributions, or to share what you've built, visit the GitHub repository.

**Keep creating. Keep learning. Keep pushing the boundaries of what's possible.**

## 5. Project Ideas

Now that you've mastered the fundamentals, here are comprehensive project ideas to deepen your skills.

### Beginner Projects (1-2 weeks each)

**1. Software Rasterizer**
- Implement triangle rasterization from scratch
- Add texture mapping with bilinear filtering
- Z-buffer for depth testing
- Flat, Gouraud, and Phong shading
- Simple camera controls

**Skills**: Scanline algorithms, interpolation, coordinate systems

**2. Whitted Ray Tracer**
- Recursive ray tracing (reflection + refraction)
- Multiple primitives (sphere, plane, triangle)
- Shadows and multiple lights
- Simple materials (diffuse, metal, glass)
- Anti-aliasing

**Skills**: Ray-primitive intersection, recursive algorithms, optics

**3. Particle System**
- Physics simulation (Euler or Verlet integration)
- Multiple emitters and behaviors
- Collisions with simple primitives
- Effects: fire, smoke, water, fireworks
- Render as sprites or instanced geometry

**Skills**: Physics, numerical integration, instancing

### Intermediate Projects (3-4 weeks each)

**4. Path Tracer**
- Monte Carlo path tracing
- Importance sampling (cosine-weighted hemisphere)
- Multiple importance sampling
- BRDFs: Lambertian, Cook-Torrance, Disney
- HDR environment maps
- Progressive rendering

**Skills**: Monte Carlo integration, probability, physically based rendering

**5. Real-Time Renderer**
- Deferred shading pipeline
- PBR materials (albedo, normal, roughness, metallic)
- Image-based lighting (IBL)
- Shadow mapping with PCF
- Post-processing (bloom, tone mapping, FXAA)

**Skills**: GPU programming, render passes, modern graphics pipelines

**6. Mesh Processor**
- OBJ file import/export
- Mesh simplification (quadric error metrics)
- Subdivision surfaces (Catmull-Clark, Loop)
- Normal computation
- UV unwrapping basics
- Boolean operations (CSG)

**Skills**: Data structures, computational geometry, mesh algorithms

**7. Animation System**
- Skeletal animation hierarchy
- Linear blend skinning (LBS)
- Keyframe interpolation (SLERP for rotations)
- Inverse kinematics (CCD or FABRIK)
- Animation blending
- Simple physics (ragdoll)

**Skills**: Hierarchical transforms, quaternions, IK algorithms

### Advanced Projects (6-8 weeks each)

**8. Production Ray Tracer**
- Full path tracer with all features
- BVH acceleration (SAH splits)
- Many primitives (triangles, curves, volumes)
- Advanced materials (SSS, volumetrics, anisotropy)
- Bidirectional path tracing
- Photon mapping for caustics
- Denoising (à-trous or AI-based)
- Scene description language
- Command-line interface with all options

**Skills**: Software engineering, optimization, production pipelines

**9. Real-Time Hybrid Engine**
- Rasterization for G-buffer
- Ray tracing for reflections, shadows, AO, GI
- Temporal accumulation
- SVGF denoising
- ReSTIR for many lights
- DLSS-style upsampling
- Performance targeting 60 FPS

**Skills**: Real-time graphics, hybrid rendering, temporal algorithms

**10. GPU Ray Tracer (CUDA/OptiX)**
- CUDA kernels for ray tracing
- BVH on GPU
- Efficient memory layouts
- Warp efficiency optimization
- Multi-GPU support
- Interactive camera
- 30+ FPS at 1080p

**Skills**: GPU programming, parallel algorithms, CUDA optimization

**11. Volume Renderer**
- Ray marching through volumes
- Participating media (fog, smoke)
- Volumetric lighting (light shafts)
- Multiple scattering
- Heterogeneous volumes (fire, clouds)
- Subsurface scattering

**Skills**: Volume rendering, scattering theory, integration

**12. Scene Editor & Renderer**
- Interactive 3D viewport (OpenGL/Vulkan)
- Object manipulation (translate, rotate, scale)
- Material editor with preview
- Light placement
- Camera controls
- Scene save/load
- Integrated ray tracer for final renders

**Skills**: UI programming, OpenGL, application architecture

### Research/Expert Projects (8+ weeks)

**13. Neural Rendering**
- Train NeRF (Neural Radiance Fields)
- Volume rendering with neural networks
- Novel view synthesis
- Light field rendering

**14. Physically Accurate Spectral Renderer**
- Full spectral rendering (wavelength-based)
- Complex IORs
- Polarization
- Fluorescence
- Matches real-world measurements

**15. Crowd Simulation & Rendering**
- Thousands of animated characters
- Level-of-detail (LOD) system
- Instanced skinned rendering
- Occlusion culling
- Procedural animation variation

### Smaller Feature Additions

Add these to any renderer:

- **Depth of field** (Chapter 10)
- **Motion blur** (Chapter 10)
- **Ambient occlusion** (Chapter 14)
- **Normal mapping** (Chapter 7)
- **HDR + tone mapping** (Chapter 17)
- **Bloom effect** (Chapter 14)
- **Procedural textures** (Perlin noise, Worley)
- **Area lights** (soft shadows)
- **Environment maps** (IBL)
- **Fog/atmosphere**
- **Lens effects** (chromatic aberration, vignette)
- **Caustics** (photon mapping)

### Learning Path Recommendations

**For Game Development**:
1. Software Rasterizer → Real-Time Renderer → Hybrid Engine
2. Focus on: Deferred shading, PBR, shadow mapping, post-processing

**For Film/VFX**:
1. Ray Tracer → Path Tracer → Production Renderer
2. Focus on: Global illumination, advanced materials, volumes, denoising

**For Research**:
1. Path Tracer → GPU Ray Tracer → Neural Rendering
2. Focus on: Novel algorithms, optimization, publication quality

**For General Graphics Programming**:
1. Software Rasterizer → Ray Tracer → Path Tracer → Animation System
2. Breadth across all topics

In [ ]:
# Create scene
scene = create_demo_scene()

# Create renderer (smaller resolution for faster rendering)
width, height = 400, 225
renderer = IntegratedRenderer(width, height)

# Render with 4 samples per pixel
image = renderer.render(scene, samples_per_pixel=4)

# Display
plt.figure(figsize=(14, 8))
plt.imshow(image)
plt.title('Complete Integrated Renderer\n(Ray Tracing + Shadows + Reflection + Multiple Materials)', 
          fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("RENDERING FEATURES DEMONSTRATED:")
print("="*70)
print("✓ Ray-sphere intersection")
print("✓ Recursive ray tracing (reflection)")
print("✓ Shadow rays (hard shadows)")
print("✓ Multiple material types (Lambertian, Metal)")
print("✓ Multiple light sources")
print("✓ Phong-like shading model")
print("✓ Anti-aliasing (4 SPP with jittering)")
print("✓ Gamma correction")
print("✓ Sky background gradient")
print("="*70)

## 4. Render the Scene

Now let's render our complete integrated scene!

In [ ]:
def create_demo_scene() -> Scene:
    """Create a demonstration scene with various materials"""
    scene = Scene()
    
    # Ground (large sphere)
    ground_mat = Material(
        type=MaterialType.LAMBERTIAN,
        albedo=Vec3(0.5, 0.5, 0.5)
    )
    scene.add_sphere(Vec3(0, -1000, 0), 1000, ground_mat)
    
    # Center sphere (diffuse red)
    center_mat = Material(
        type=MaterialType.LAMBERTIAN,
        albedo=Vec3(0.7, 0.3, 0.3)
    )
    scene.add_sphere(Vec3(0, 1, 0), 1.0, center_mat)
    
    # Left sphere (metal)
    left_mat = Material(
        type=MaterialType.METAL,
        albedo=Vec3(0.8, 0.8, 0.8),
        roughness=0.3
    )
    scene.add_sphere(Vec3(-2.5, 1, 0), 1.0, left_mat)
    
    # Right sphere (gold metal)
    right_mat = Material(
        type=MaterialType.METAL,
        albedo=Vec3(0.8, 0.6, 0.2),
        roughness=0.1
    )
    scene.add_sphere(Vec3(2.5, 1, 0), 1.0, right_mat)
    
    # Small spheres in front
    small_mat1 = Material(
        type=MaterialType.LAMBERTIAN,
        albedo=Vec3(0.1, 0.5, 0.8)
    )
    scene.add_sphere(Vec3(-1, 0.5, 2), 0.5, small_mat1)
    
    small_mat2 = Material(
        type=MaterialType.LAMBERTIAN,
        albedo=Vec3(0.9, 0.9, 0.1)
    )
    scene.add_sphere(Vec3(1, 0.5, 2), 0.5, small_mat2)
    
    # Lights
    scene.add_light(
        position=Vec3(-5, 10, -5),
        color=Vec3(1, 1, 1),
        intensity=0.8
    )
    
    scene.add_light(
        position=Vec3(5, 5, 5),
        color=Vec3(1, 0.9, 0.8),
        intensity=0.5
    )
    
    # Camera
    look_from = Vec3(8, 3, 8)
    look_at = Vec3(0, 1, 0)
    vup = Vec3(0, 1, 0)
    vfov = 40.0
    aspect = 16.0 / 9.0
    
    camera = Camera(look_from, look_at, vup, vfov, aspect)
    scene.set_camera(camera)
    
    return scene

print("✓ Scene creation function loaded")

## 3. Example Scene Setup

Let's create a complete scene with multiple materials and lighting.

In [ ]:
class IntegratedRenderer:
    """Complete renderer with all features"""
    
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.framebuffer = np.zeros((height, width, 3))
        self.max_depth = 5
    
    def reflect(self, v: Vec3, n: Vec3) -> Vec3:
        """Reflect vector v about normal n"""
        return v - n * (2.0 * v.dot(n))
    
    def schlick(self, cosine: float, ior: float) -> float:
        """Schlick's approximation for Fresnel"""
        r0 = ((1 - ior) / (1 + ior)) ** 2
        return r0 + (1 - r0) * ((1 - cosine) ** 5)
    
    def shade_lambertian(self, hit: HitRecord, scene: Scene, material: Material) -> Vec3:
        """Lambertian (diffuse) shading"""
        color = Vec3(0, 0, 0)
        
        for light in scene.lights:
            # Direction to light
            light_dir = (light.position - hit.point).normalize()
            
            # Shadow ray
            shadow_ray = Ray(hit.point, light_dir)
            shadow_hit = scene.intersect(shadow_ray)
            
            # Check if in shadow
            dist_to_light = (light.position - hit.point).length()
            if shadow_hit.hit and shadow_hit.t < dist_to_light:
                continue  # In shadow
            
            # Diffuse term
            diffuse = max(0.0, hit.normal.dot(light_dir))
            
            # Accumulate light contribution
            color = color + Vec3(
                material.albedo.x * light.color.x * diffuse * light.intensity,
                material.albedo.y * light.color.y * diffuse * light.intensity,
                material.albedo.z * light.color.z * diffuse * light.intensity
            )
        
        # Ambient
        ambient = Vec3(0.1, 0.1, 0.1)
        color = color + Vec3(
            material.albedo.x * ambient.x,
            material.albedo.y * ambient.y,
            material.albedo.z * ambient.z
        )
        
        return color
    
    def shade_metal(self, ray: Ray, hit: HitRecord, scene: Scene, material: Material, depth: int) -> Vec3:
        """Metal (reflective) shading"""
        reflected = self.reflect(ray.direction, hit.normal)
        reflected_ray = Ray(hit.point, reflected)
        
        # Recursive reflection
        reflected_color = self.trace_ray(reflected_ray, scene, depth + 1)
        
        return Vec3(
            material.albedo.x * reflected_color.x,
            material.albedo.y * reflected_color.y,
            material.albedo.z * reflected_color.z
        )
    
    def trace_ray(self, ray: Ray, scene: Scene, depth: int = 0) -> Vec3:
        """Trace a ray through the scene"""
        
        # Max recursion depth
        if depth >= self.max_depth:
            return Vec3(0, 0, 0)
        
        # Intersect scene
        hit = scene.intersect(ray)
        
        if not hit.hit:
            # Background color (sky)
            t = 0.5 * (ray.direction.y + 1.0)
            return Vec3(
                (1.0 - t) * 1.0 + t * scene.background.x,
                (1.0 - t) * 1.0 + t * scene.background.y,
                (1.0 - t) * 1.0 + t * scene.background.z
            )
        
        # Get material
        material = scene.materials[hit.material_id]
        
        # Emission (for lights)
        color = material.emission
        
        # Shade based on material type
        if material.type == MaterialType.LAMBERTIAN:
            color = color + self.shade_lambertian(hit, scene, material)
        elif material.type == MaterialType.METAL:
            color = color + self.shade_metal(ray, hit, scene, material, depth)
        
        return color
    
    def render(self, scene: Scene, samples_per_pixel: int = 1) -> np.ndarray:
        """Render the scene"""
        if scene.camera is None:
            raise ValueError("Scene must have a camera")
        
        print(f"Rendering {self.width}x{self.height} at {samples_per_pixel} SPP...")
        
        for y in range(self.height):
            if y % 20 == 0:
                print(f"  Scanline {y}/{self.height}")
            
            for x in range(self.width):
                color = Vec3(0, 0, 0)
                
                # Multi-sampling
                for _ in range(samples_per_pixel):
                    # Normalized coordinates with jitter
                    u = (x + random.random()) / (self.width - 1)
                    v = 1.0 - (y + random.random()) / (self.height - 1)
                    
                    ray = scene.camera.get_ray(u, v)
                    sample_color = self.trace_ray(ray, scene)
                    color = color + sample_color
                
                # Average samples
                color = color * (1.0 / samples_per_pixel)
                
                # Gamma correction
                color = Vec3(
                    math.sqrt(max(0, color.x)),
                    math.sqrt(max(0, color.y)),
                    math.sqrt(max(0, color.z))
                )
                
                # Clamp and store
                self.framebuffer[y, x] = [
                    min(1.0, color.x),
                    min(1.0, color.y),
                    min(1.0, color.z)
                ]
        
        print("✓ Rendering complete!")
        return self.framebuffer

print("✓ Integrated renderer loaded")

## 2. Complete Renderer Implementation

Now let's build a complete renderer that integrates all techniques:
- Ray tracing with reflection
- Phong shading
- Shadows
- Multiple materials

## Acknowledgments

This tutorial was built from first principles using Python, NumPy, and Matplotlib. All implementations are educational and designed to run in Google Colab for accessibility.

The content covers fundamental and advanced topics in 3D computer graphics, suitable for students, researchers, and practitioners interested in understanding rendering from the ground up.

For questions, improvements, or contributions, please visit the GitHub repository.